In [49]:
import sys
sys.path.append('C:/Automations/5/App')

import io
import re
import sqlite3
import requests
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from pandas import DataFrame, DateOffset

from Modules.pdf import Pdf
from Config.folders import ONEDRIVE

In [50]:
def get_last_friday(date: datetime) -> datetime:
    """Returns the date of the previous friday."""
    offset = 0
    while (get_last_friday := date - timedelta(days = offset)).weekday() != 4:
        offset += 1

    return datetime(get_last_friday.year, get_last_friday.month, get_last_friday.day)

In [51]:
def sum_year(current_date: datetime, years: int) -> datetime:
    return datetime.fromisoformat((current_date + DateOffset(years = years)).isoformat())

In [52]:
date = datetime(2022, 12, 30)

In [71]:
# Adaptation of https://stackoverflow.com/questions/59905947/pdfplumber-to-handle-pdf-online
file = io.BytesIO(requests.get(f"https://www.bcb.gov.br/content/focus/focus/R20220909.pdf").content)

In [82]:
content = Pdf(file).read_text().split('\n')

In [ ]:
year1 = list()
year2 = list()
year3 = list()
year4 = list()

for line in content:
    # Breakthrough: https://regex101.com/r/O4EOaf/1
    if (matches := re.match(r"^(IPCA \(variação %\)|PIB Total \(variação % sobre ano anterior\)|Câmbio \(R\$\/US\$\)|Selic \(% a\.a\)|IGP-M \(variação %\)|IPCA Administrados \(variação %\)|Conta corrente \(US\$ bilhões\)|Balança comercial \(US\$ bilhões\)|Investimento direto no país \(US\$ bilhões\)|Dívida líquida do setor público \(% do PIB\)|Resultado primário \(% do PIB\)|Resultado nominal \(% do PIB\)) (([-\d,]+|-) ([-\d,]+|-) ([-\d,]+|-)( \W \(\d\)| \(\d\))?( [\d]+)?( [-\d,]+|-)?( [\d]+)?){1} (([-\d,]+|-) ([-\d,]+|-) ([-\d,]+|-)( \W \(\d\)| \(\d\))?( [\d]+)?( [-\d,]+|-)?( [\d]+)?){1} (([-\d,]+|-) ([-\d,]+|-) ([-\d,]+|-)( \W \(\d+\)| \(\d+\))?( [\d]+)?){1} (([-\d,]+|-) ([-\d,]+|-) ([-\d,]+|-)( \W \(\d+\)| \(\d+\))?( [\d]+)?){1}$", line)):

        year1.append({
            "Data Referencia": date
            ,"Ano Referencia": date
            ,"Indicador": matches.group(1) 
            ,"4 Semanas": matches.group(3) if matches.group(3) != "-" else (np.nan)
            ,"1 Semana": matches.group(4) if matches.group(4) != "-" else (np.nan)
            ,"Hoje": matches.group(5) if matches.group(5) != "-" else (np.nan)
            ,"Respondentes 30 Dias": matches.group(7) if matches.group(7) else (np.nan)
            ,"5 Dias Uteis": matches.group(8) if matches.group(8) else (np.nan)
            ,"Respondentes 5 Dias": matches.group(9) if matches.group(9) else (np.nan)
        })

        year2.append({
            "Data Referencia": date
            ,"Ano Referencia": sum_year(get_last_friday(date), 1)
            ,"Indicador": matches.group(1) 
            ,"4 Semanas": matches.group(11) if matches.group(11) != "-" else (np.nan)
            ,"1 Semana": matches.group(12) if matches.group(12) != "-" else (np.nan)
            ,"Hoje": matches.group(13) if matches.group(13) != "-" else (np.nan)
            ,"Respondentes 30 Dias": matches.group(15) if matches.group(15) else (np.nan)
            ,"5 Dias Uteis": matches.group(16) if matches.group(16) else (np.nan)
            ,"Respondentes 5 Dias": matches.group(17) if matches.group(17) else (np.nan)
        })
        
        year3.append({
            "Data Referencia": date
            ,"Ano Referencia": sum_year(get_last_friday(date), 2)
            ,"Indicador": matches.group(1)
            ,"4 Semanas": matches.group(19) if matches.group(19) != "-" else (np.nan)
            ,"1 Semana": matches.group(20) if matches.group(20) != "-" else (np.nan)
            ,"Hoje": matches.group(21) if matches.group(21) != "-" else (np.nan)
            ,"Respondentes 30 Dias": matches.group(23) if matches.group(23) != "-" else (np.nan)
        })
        
        year4.append({
            "Data Referencia": get_last_friday(date)
            ,"Ano Referencia": sum_year(get_last_friday(date), 3)
            ,"Indicador": matches.group(1)
            ,"4 Semanas": matches.group(25) if matches.group(25) != "-" else (np.nan)
            ,"1 Semana": matches.group(26) if matches.group(26) != "-" else (np.nan)
            ,"Hoje": matches.group(27) if matches.group(27) != "-" else (np.nan)
            ,"Respondentes 30 Dias": matches.group(29) if matches.group(29) != "-" else (np.nan)
        })

year1 = DataFrame(year1)
year2 = DataFrame(year2)
year3 = DataFrame(year3)
year4 = DataFrame(year4)

In [ ]:
FatoExpectativasBCB: DataFrame = pd.concat([year1, year2, year3, year4], ignore_index = True).replace("-", np.nan)

In [ ]:
FatoExpectativasBCB[["4 Semanas", "1 Semana", "Hoje", "5 Dias Uteis"]] = FatoExpectativasBCB[["4 Semanas", "1 Semana", "Hoje", "5 Dias Uteis"]].map(lambda number: str(number).replace(",", "."))

In [ ]:
FatoExpectativasBCB = FatoExpectativasBCB.astype(
    dtype = {
        "Data Referencia": "datetime64[ns]"
        ,"Ano Referencia": "datetime64[ns]"
        ,"Indicador": str
        ,"4 Semanas": "float64"
        ,"1 Semana": "float64"
        ,"Hoje": "float64"
        ,"Respondentes 30 Dias": "float64"
        ,"5 Dias Uteis": "float64"
        ,"Respondentes 5 Dias": "float64"
    }
)

In [ ]:
FatoExpectativasBCB

,Data Referencia,Ano Referencia,Indicador,4 Semanas,1 Semana,Hoje,Respondentes 30 Dias,5 Dias Uteis,Respondentes 5 Dias
0,2019-03-12,2019-03-12,IPCA (variação %),5.92,5.64,5.62,139.0,5.64,35.0
1,2019-03-12,2019-03-12,PIB Total (variação % sobre ano anterior),3.05,3.04,3.04,98.0,3.00,22.0
2,2019-03-12,2019-03-12,Câmbio (R$/US$),5.25,5.25,NaN,NaN,NaN,NaN
3,2019-03-12,2019-03-12,Selic (% a.a),13.75,NaN,NaN,NaN,NaN,NaN
4,2019-03-12,2019-03-12,IGP-M (variação %),5.42,5.51,NaN,NaN,NaN,NaN
5,2019-03-12,2019-03-12,IPCA Administrados (variação %),-3.59,-3.86,-3.89,88.0,-3.90,21.0
6,2019-03-12,2019-03-12,Conta corrente (US$ bilhões),-46.20,-51.00,-51.30,25.0,-55.00,7.0
7,2019-03-12,2019-03-12,Balança comercial (US$ bilhões),55.00,56.95,56.90,23.0,50.00,5.0
8,2019-03-12,2019-03-12,Investimento direto no país (US$ bilhões),78.00,81.60,81.60,22.0,78.55,6.0
9,2019-03-12,2019-03-12,Dívida líquida do setor público (% do PIB),57.70,57.50,57.50,22.0,58.00,5.0


In [ ]:
database = pd.read_sql("SELECT * FROM FatoExpectativasBCB", sqlite3.connect(ONEDRIVE / "Manchester - Mesa RV - Backoffice - 0/Dados/economia.db"))

In [ ]:
FatoExpectativasBCB[~FatoExpectativasBCB["Data Referencia"].isin(database["Data Referencia"])]

C:\Users\Manchester\AppData\Local\Temp\ipykernel_18620\785314427.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  FatoExpectativasBCB[~FatoExpectativasBCB["Data Referencia"].isin(database["Data Referencia"])]


,Data Referencia,Ano Referencia,Indicador,4 Semanas,1 Semana,Hoje,Respondentes 30 Dias,5 Dias Uteis,Respondentes 5 Dias


In [ ]:
dates = list()
offset = 7
for _ in range(208):
    dates.append(datetime(2023, 3, 7) - timedelta(days = offset))
    offset += 7

In [ ]:
links = list()
for date in dates:
    links.append(f"https://www.bcb.gov.br/content/focus/focus/R{get_last_friday(date).strftime("%Y%m%d")}.pdf")

In [ ]:
links

['https://www.bcb.gov.br/content/focus/focus/R20230224.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230217.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230210.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230203.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230127.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230120.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230113.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20230106.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221230.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221223.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221216.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221209.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221202.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221125.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221118.pdf',
 'https://www.bcb.gov.br/content/focus/focus/R20221111.pdf',
 'https://www.bcb.gov.br